# <center> <b> Zomato Interactive Dashboard </b> </center>

#### Bengaluru Is India's IT Capital. The city with a huge number of tech firms is known as India's Silicon Valley. Many of the students and workers have shifted in Bengaluru due to their work and they are heavily reliant on the restaurant food. The goal of the project is to analyze the Zomato Bengaluru dataset in order to get a good understanding of the factors that affect the restaurants in Bengaluru. This can also can help new start-up restaurants determine their location, restaurant type, menus, cuisine, cost of the dishes etc.


<center> <img src="https://upload.wikimedia.org/wikipedia/commons/e/ef/Zomato-flat-logo.png" width=500 height=500 /> </center>

In [ ]:
import numpy as np 
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from ipywidgets import interact
from google_drive_downloader import GoogleDriveDownloader as gdd
import ipywidgets as widgets
import folium
from folium.plugins import HeatMap
import wordcloud
from wordcloud import WordCloud

In [ ]:
#To deploy it globally, follow the steps:
# 1. You need to upload the dataset on GitHub or on GoogleDrive
# 2. You need a runtime.txt file that specifies the python version (make sure the spelling is not changed)
# 3. You need a requirements.txt file that contains all the libraries used by you. (make sure the spelling is not changed)
# 4. You need a Procfile (with no extensions) that contains: 'web: voila --port=$PORT --no-browser --theme=dark Session_3_Global_Deploy.ipynb'  (make sure the spelling is not changed)

In [ ]:
gdd.download_file_from_google_drive(file_id='1SBJDhpzTpf4Y2d8CVs9OdjWPufUN492j',
                                    dest_path='./Zom_cleaned.csv',
                                    unzip=False)
gdd.download_file_from_google_drive(file_id='1tHi7mY0QyXxcYuh3sxjLr8a4yWmfMRw0',
                                    dest_path='./locations.csv',
                                    unzip=False)

In [ ]:
df=pd.read_csv("Zom_cleaned.csv")
df= df.drop(['reviews_list','listed_in(city)'],1)

# <center> <b> Get the details of the desired restaurant </b> </center>

In [ ]:
pd.set_option('max_colwidth', None)

In [ ]:
res=df['name'].unique().tolist()
def rest(Restaurant_Name):
  res_de = df[(df.name == Restaurant_Name)].drop('url', axis=1)
  return res_de
interact(rest, Restaurant_Name=res);

interactive(children=(Dropdown(description='Restaurant_Name', options=('Jalsa', 'Spice Elephant', 'San Churro …

# <center> <b> Type your Bangalore location and get highly rated budget-friendly restaurant options! </b> </center>

In [ ]:
cost_dist=df[['rate','approx_cost(for two people)','location','name']].dropna()
rb =cost_dist['location'].unique().tolist()

In [ ]:
def return_budget(location):
    budget=cost_dist[(cost_dist['approx_cost(for two people)']<=400) & (cost_dist['location']==location) & (cost_dist['rate']>4)]
    budget=budget['name'].unique()
    budget= pd.DataFrame(budget, columns=['Restaurants'])
    return (budget)
interact(return_budget, location= rb);

interactive(children=(Dropdown(description='location', options=('Banashankari', 'Basavanagudi', 'Mysore Road',…

# <center> <b> Which are the top restaurant chains in Bangalore? </b> </center>

In [ ]:
def top(n):
  chain = df['name'].value_counts().head(n).to_frame()
  fig = px.bar(chain, x = chain.index, y = 'name', color='name',labels={'name':'count','x':'Restaurant'})
  fig.show();
interact(top, n=10);

interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

# <center> <b> Heatmap of restaurants in Bangalore <b/> </center>

In [ ]:
loc = pd.read_csv("locations.csv")

In [ ]:
loc["Name"]=loc['Name'].apply(lambda x :  x.replace("Bangalore","")[1:])
Rest_locations=pd.DataFrame(df['location'].value_counts().reset_index())
Rest_locations.columns=['Name','count']
Rest_locations=Rest_locations.merge(loc,on='Name',how="left").dropna()
Rest_locations['lat'], Rest_locations['lon'] = Rest_locations['geo_loc'].str.split(',', 1).str
Rest_locations['lat'] = Rest_locations['lat'].apply(lambda x: x.replace('(','')) 
Rest_locations['lon'] = Rest_locations['lon'].apply(lambda x: x.replace(')','')) 
Rest_locations['lat']=Rest_locations['lat'].astype(float)
Rest_locations['lon']=Rest_locations['lon'].astype(float)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning:

Columnar iteration over characters will be deprecated in future releases.



In [ ]:
def generateBaseMap(default_location=[12.97, 77.59], default_zoom_start=12):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map
basemap=generateBaseMap();
HeatMap(Rest_locations[['lat','lon','count']].values.tolist()).add_to(basemap);
basemap

# <center> <b> Rating Distribution </b> </center>

In [ ]:
rating=df['rate'].dropna()
rating = rating.value_counts().to_frame()
fig = px.bar(rating, x = rating.index, y = rating.rate, color= rating.rate,labels={'rate':'count','x':'Rating'})
fig.show();

# <center> <b> Top Foodie Areas in Bangalore <b/> </center>

In [ ]:
def top_area(n):
    foodie = df['location'].value_counts().head(n).to_frame()
    fig=px.bar(foodie, x = foodie.index, y = 'location', color='location',labels={'location':'count','x':'location'})
    fig.show();
interact(top_area, n=5);

interactive(children=(IntSlider(value=5, description='n', max=15, min=-5), Output()), _dom_classes=('widget-in…

# <center> <b> Restaurant ratings v/s Cost for two (hover over each data point for more details) <b/> </center>

In [ ]:
d = df.dropna()
d = d.sort_values(by='rate',ascending = True)
trace = go.Scatter(y = d['rate'], x = d['approx_cost(for two people)'], text = d['name'], mode = 'markers')
layout = go.Layout(title='Cost v/s Ratings', xaxis = dict(title='Cost for two'), yaxis = dict(title='Restaurant Rating'))
go.Figure(data = trace , layout = layout)

# <center> <b> Top Dishes liked in Bangalore <b/> </center>

In [ ]:
wc = df['rest_type'].unique().tolist()

In [ ]:
def show_wordcloud(Restaurant):
    plt.figure(figsize=(20,30))
    df_dish = df.dropna(subset=['dish_liked'])
    text = ' '.join(df_dish[df_dish['rest_type']==Restaurant]['dish_liked'].astype(str).tolist())
    fig_wordcloud = WordCloud(max_words=40, 
                    background_color='white', width=1500, height=800, random_state=1).generate(text)
    
    plt.imshow(fig_wordcloud)  
    plt.title(Restaurant, fontsize=20)
    plt.show()
interact(show_wordcloud, Restaurant=wc);

interactive(children=(Dropdown(description='Restaurant', options=('Casual Dining', 'Cafe, Casual Dining', 'Qui…

# <center> <b> Heatmap as per Cuisine <b/> </center>

In [ ]:
cui = df['cuisines'].unique().tolist()

In [ ]:
def get_cui(Cuisine):
    data= pd.DataFrame(df[df['cuisines']==Cuisine].groupby(['location'],as_index=False)['url'].agg('count'))
    data.columns=['Name','count']
    data = data.merge(loc,on="Name",how='left').dropna()
    data['lat'], data['lon'] = data['geo_loc'].str.split(',', 1).str
    data['lat'] = data['lat'].apply(lambda x: x.replace('(','')) 
    data['lon'] = data['lon'].apply(lambda x: x.replace(')',''))
    data['lat'] = data['lat'].astype(float)
    data['lon'] = data['lon'].astype(float)
    base_map = folium.Map(location=[12.97, 77.59], control_scale=True, zoom_start=12)
    HeatMap(data[['lat','lon','count']].values.tolist()).add_to(base_map)
    return base_map
interact(get_cui, Cuisine=cui);

interactive(children=(Dropdown(description='Cuisine', options=('North Indian, Mughlai, Chinese', 'Chinese, Nor…